In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import re
import time

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("review_data.csv")
df.head(20)

,score,review,y
0,5.0,명불허전,1
1,5.0,"이거슨 한국의 까르보나라,,,🤍 들깨칼국수 고소하니 넘 맛나유 얼큰해물칼국수는 해물...",1
2,5.0,들깨가 찐하고 고소해요. 음식맛이 깔끔해요. 어르신 모시고 청주 온다면 식사하기 괜...,1
3,4.0,청주 사람들 점심엔 여기만 오나봐요. 주말 점심시간에 왔는데 손님 100명은 앉아있...,1
4,5.0,NaN,1
5,5.0,NaN,1
6,5.0,4명에서 3~4인 셋트시키면 진짜 배터지게 먹고나올수있어요!,1
7,5.0,들깨칼국수 땡기면 찾아가는곳,1
8,5.0,일하시는분들이 친절하고 음식이 들깨칼국수 맛있어요,1
9,3.0,NaN,0
